In [16]:
import pandas as pd
from sqlalchemy import create_engine

import sys
path = r"D:\JupyterProject\跨境统计"
sys.path.append(path)
import func_new

database = 'ec_cross_border'
host = '192.168.0.227'
keys = 'Data227or8Dev715#'
user = 'dev'

In [2]:
engine_str = 'mysql+pymysql://{user}:{pw}@{host}/{db}?charset=utf8'
engine = create_engine(engine_str.format(user=user,pw=keys,host=host,db=database),encoding='utf-8')

In [3]:
date = '202002'

In [4]:
shop = pd.read_sql("select seller_name,country from ebay_usrinfo_{0}".format(date),con=engine)

In [5]:
#不统计乱码的（销量是整数，不会带小数点）
goods_all= pd.read_sql("select good_name,price_dollar,project_location,seller_name,sales_count,brand,cat_1 from ebay_goodsinfo_{0} where sales_count not like'%%.%%' and price_dollar not like '%% %%'".format(date),con=engine)

In [6]:
goods_all.loc[goods_all.price_dollar=='.','price_dollar']=0

In [17]:
goods_all = func.change_data_type(goods_all,['price_dollar','sales_count'],['float','int'])

ValueError: could not convert string to float: '.'

In [ ]:
goods_all['sales_money'] = goods_all.price_dollar* goods_all.sales_count

In [ ]:
goods_all.to_csv("ebay{0}商品数据.csv".format(date),index=False)

In [ ]:
goods = goods_all.groupby(['project_location','seller_name'])[['sales_count','sales_money']].sum().reset_index()

In [ ]:
data =pd.merge(shop,goods)

In [ ]:
country = data.groupby('country')[['sales_count','sales_money']].sum().reset_index()

country.to_excel('国家.xlsx',index=False)

In [ ]:
def change_country_name(term):
    if term.country=='China':
        term.country = '中国'
    elif term.country=='Hong Kong':
        term.country = '香港'
    elif term.country=='India':
        term.country = '印度'
    elif term.country == 'United Kingdom':
        term.country = '英国'
    elif term.country == 'United States':
        term.country = '美国'
    return term

In [ ]:
country = country.apply(change_country_name,axis=1)

In [ ]:
country = country.groupby('country')[['sales_money','sales_count']].sum().reset_index()

In [ ]:
country_columns_rename={'country':'国家','sales_money':'销售额','sales_count':'销售量'}

In [ ]:
country.rename(columns=country_columns_rename).sort_values('销售额',ascending=False).to_excel("ebay{0}国家统计数据.xlsx".format(date),index=False)